In [1]:
import requests
import pyarrow as pa
import pyarrow.fs
import io
import os

In [2]:
#q1
!hdfs dfsadmin -fs hdfs://boss:9000/ -report

Configured Capacity: 51642105856 (48.10 GB)
Present Capacity: 15915597824 (14.82 GB)
DFS Remaining: 15915548672 (14.82 GB)
DFS Used: 49152 (48 KB)
DFS Used%: 0.00%
Replicated Blocks:
	Under replicated blocks: 0
	Blocks with corrupt replicas: 0
	Missing blocks: 0
	Missing blocks (with replication factor 1): 0
	Low redundancy blocks with highest priority to recover: 0
	Pending deletion blocks: 0
Erasure Coded Block Groups: 
	Low redundancy block groups: 0
	Block groups with corrupt internal blocks: 0
	Missing block groups: 0
	Low redundancy blocks with highest priority to recover: 0
	Pending deletion blocks: 0

-------------------------------------------------
Live datanodes (2):

Name: 172.22.0.3:9866 (project-4-team_bds_p4-dn-1.project-4-team_bds_p4_default)
Hostname: 952ff983a1b1
Decommission Status : Normal
Configured Capacity: 25821052928 (24.05 GB)
DFS Used: 24576 (24 KB)
Non DFS Used: 17846476800 (16.62 GB)
DFS Remaining: 7957774336 (7.41 GB)
DFS Used%: 0.00%
DFS Remaining%: 30.82

In [3]:
os.system("rm *.csv*")
!wget https://pages.cs.wisc.edu/~harter/cs544/data/hdma-wi-2021.csv

--2023-10-28 00:19:18--  https://pages.cs.wisc.edu/~harter/cs544/data/hdma-wi-2021.csv
Resolving pages.cs.wisc.edu (pages.cs.wisc.edu)... 128.105.7.9
Connecting to pages.cs.wisc.edu (pages.cs.wisc.edu)|128.105.7.9|:443... connected.
HTTP request sent, awaiting response... 

rm: cannot remove '*.csv*': No such file or directory


200 OK
Length: 174944099 (167M) [text/csv]
Saving to: ‘hdma-wi-2021.csv’

hdma-wi-2021.csv    100%[===================>] 166.84M  77.8MB/s    in 2.1s    

2023-10-28 00:19:20 (77.8 MB/s) - ‘hdma-wi-2021.csv’ saved [174944099/174944099]



In [4]:
!hdfs dfs -rm -f hdfs://boss:9000/single.csv
!hdfs dfs -rm -f hdfs://boss:9000/double.csv

In [6]:
!hdfs dfs -D dfs.block.size=1048576 -D dfs.replication=1 -cp hdma-wi-2021.csv hdfs://boss:9000/single.csv
!hdfs dfs -D dfs.block.size=1048576 -D dfs.replication=2 -cp hdma-wi-2021.csv hdfs://boss:9000/double.csv

cp: `hdfs://boss:9000/single.csv': File exists
cp: `hdfs://boss:9000/double.csv': File exists


In [7]:
#q2
!hdfs dfs -du -s -h hdfs://boss:9000/single.csv
!hdfs dfs -du -s -h hdfs://boss:9000/double.csv

166.8 M  166.8 M  hdfs://boss:9000/single.csv
166.8 M  333.7 M  hdfs://boss:9000/double.csv


In [8]:
#q3
url_s = f"http://boss:9870/webhdfs/v1/single.csv?op=GETFILESTATUS"
r_single = requests.get(url_s)
r_single.json()

{'FileStatus': {'accessTime': 1698452388515,
  'blockSize': 1048576,
  'childrenNum': 0,
  'fileId': 16386,
  'group': 'supergroup',
  'length': 174944099,
  'modificationTime': 1698452394578,
  'owner': 'root',
  'pathSuffix': '',
  'permission': '644',
  'replication': 1,
  'storagePolicy': 0,
  'type': 'FILE'}}

In [9]:
#q4
r = requests.get(f"http://boss:9870/webhdfs/v1/single.csv?op=OPEN&offset=0&noredirect=true")
r.json()['Location']

'http://952ff983a1b1:9864/webhdfs/v1/single.csv?op=OPEN&namenoderpcaddress=boss:9000&offset=0'

In [10]:
#q5
r_total = requests.get(f"http://boss:9870/webhdfs/v1/single.csv?op=GETFILESTATUS")
total_size = r_total.json()['FileStatus']['length']
block_size = r_total.json()['FileStatus']['blockSize']
blocks_count = {}
i = 0
while(i*block_size < total_size):
    url = "http://boss:9870/webhdfs/v1/single.csv?op=OPEN&offset={}&noredirect=true".format(block_size*i)
    r0 = requests.get(url)
    info = r0.json()
    host = info['Location'].split("/")[2][:-5]
    if host in blocks_count:
        blocks_count[host] += 1
    else:
        blocks_count[host] = 1
    i+=1
blocks_count

{'952ff983a1b1': 81, '6ac7f31c9295': 86}

In [11]:
#q6
hdfs = pa.fs.HadoopFileSystem("boss", 9000)
f = hdfs.open_input_file("/single.csv")
f.read_at(10, 0)

2023-10-28 00:21:08,065 WARN util.NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


b'activity_y'

In [12]:
#q7
with hdfs.open_input_file("/single.csv") as f:
    reader = io.TextIOWrapper(io.BufferedReader(f))
    count = 0
    for line in reader:
        if "Single Family" in line:
                count += 1
count

444874